In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
audio_files_dir = '/content/gdrive/My Drive/UpworkNotebooks/Opera Files/'

!ls -ltra '{audio_files_dir}'/.

Mounted at /content/gdrive
ls: '/content/gdrive/My Drive/UpworkNotebooks/Opera Files//./Audios': No such file or directory
total 24
lrw------- 1 root root    0 May  1  2021 Audios -> /content/gdrive/.shortcut-targets-by-id/1DOvML8hO7v6XEq5-HQAQSvo7eKca7tSf/Audios
drwx------ 2 root root 4096 Sep 22 14:29 All_Data
drwx------ 2 root root 4096 Sep 22 14:57 Documentation
drwx------ 2 root root 4096 Sep 22 15:02 Helper_Notebooks
drwx------ 2 root root 4096 Sep 22 15:03 Csv_Files
drwx------ 2 root root 4096 Sep 22 15:05 Main_Notebooks
drwx------ 2 root root 4096 Dec 10 07:34 Models


In [2]:
!pip install -q librosa
!pip install -q tensorflow_io
!pip install -q pysoundfile

     |████████████████████████████████| 23.4 MB 1.5 MB/s 


In [8]:
import os
import librosa

from IPython import display
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_io as tfio

import time

%matplotlib inline

import urllib.request as urllib2 # For python3
import sklearn
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from matplotlib.colors import LogNorm

In [3]:
src_path = audio_files_dir + "All_Data/females_500_zip/females_500.zip"
dst_path = audio_files_dir + "All_Data/females_500"
print(src_path)
print(dst_path)

/content/gdrive/My Drive/UpworkNotebooks/Opera Files/All_Data/females_500_zip/females_500.zip
/content/gdrive/My Drive/UpworkNotebooks/Opera Files/All_Data/females_500


In [4]:
!unzip '{src_path}' -d '{dst_path}'

Archive:  /content/gdrive/My Drive/UpworkNotebooks/Opera Files/All_Data/females_500_zip/females_500.zip
   creating: /content/gdrive/My Drive/UpworkNotebooks/Opera Files/All_Data/females_500/Females_only_500/
   creating: /content/gdrive/My Drive/UpworkNotebooks/Opera Files/All_Data/females_500/Females_only_500/contraltos/
  inflating: /content/gdrive/My Drive/UpworkNotebooks/Opera Files/All_Data/females_500/Females_only_500/contraltos/Dolc__e_pur_d_amor_l_affanno,_HWV_109_-_Aria_-_Se_piu_non_t_amo-HpNv_DgF9xo_1.wav  
  inflating: /content/gdrive/My Drive/UpworkNotebooks/Opera Files/All_Data/females_500/Females_only_500/contraltos/Dolc__e_pur_d_amor_l_affanno,_HWV_109_-_Aria_-_Se_piu_non_t_amo-HpNv_DgF9xo_10.wav  
  inflating: /content/gdrive/My Drive/UpworkNotebooks/Opera Files/All_Data/females_500/Females_only_500/contraltos/Dolc__e_pur_d_amor_l_affanno,_HWV_109_-_Aria_-_Se_piu_non_t_amo-HpNv_DgF9xo_11.wav  
  inflating: /content/gdrive/My Drive/UpworkNotebooks/Opera Files/All_Data/f

In [19]:
#####save file_paths and their respective subregisters in .csv file####
directory = audio_files_dir + "All_Data/females_500/"
print(directory)
paths = [os.path.abspath(os.path.join(dirpath, file)) for dirpath,_,filenames in os.walk(directory) for file in filenames]
subregisters = [os.path.basename(os.path.normpath(dirpath)) for dirpath,_,filenames in os.walk(directory) for file in filenames]

print(subregisters)
print(paths)
data = pd.DataFrame()

data["path"] = paths
data["subregister"] = subregisters

classes_mapping = dict()
for num, subregister in enumerate(data["subregister"].unique()):
  classes_mapping[subregister]=num
class_id = data['subregister'].apply(lambda name: classes_mapping[name])
data = data.assign(Target=class_id)

print(data.head())

data.to_csv(audio_files_dir + "Csv_Files/females_500_paths.csv", index=False)

/content/gdrive/My Drive/UpworkNotebooks/Opera Files/All_Data/females_500/
['contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 'contraltos', 

In [20]:
data.to_csv(audio_files_dir + "Csv_Files/females_500_paths.csv", index=False)

In [12]:
print(len(data))



4829


In [21]:
data['subregister'].unique()

array(['contraltos', 'countertenor', 'mezzosoprano coloratura',
       'mezzosoprano dramatic', 'mezzosoprano lyric',
       'soprano coloratura', 'soprano dramatic', 'soprano lyric',
       'soprano soubrette', 'soprano spinto'], dtype=object)

In [14]:
def extract_features(file_name):
  audio, sample_rate = librosa.load(file_name) 
  mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=128)
  mfccs_processed = np.mean(mfccs.T,axis=0)
  mfccs_processed = list(mfccs_processed)
  return mfccs_processed

mfcc_featuresdf = pd.DataFrame()
mfcc_featuresdf["mfccs"] = list(map(extract_features, data["path"]))
mfcc_featuresdf["class_label"] = data["Target"]

mfcc_featuresdf["mfccs"]
mfcc_featuresdf.to_csv(audio_files_dir + "Csv_Files/females_500.csv", index=False)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")

In [15]:
mfcc_featuresdf["class_label"].unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [16]:
data = pd.read_csv(audio_files_dir + "Csv_Files/females_500.csv")
data.head()

,mfccs,class_label
0,"[-343.66223, 129.16498, -103.39126, -10.168028...",0
1,"[-353.95767, 131.35814, -85.355576, -4.559168,...",0
2,"[-301.94495, 118.35664, -94.49565, -11.121386,...",0
3,"[-340.73184, 138.08566, -88.15859, 5.997671, -...",0
4,"[-331.90997, 146.29329, -99.99819, 19.48146, -...",0
